In [1]:
import pandas as pd
import os
root_dir = os.getcwd()
from data_loading import load_ftn, load_stadium_data, load_weather_data
from data_cleaning import feature_engineering


#### Load data

In [3]:
df_games=pd.read_csv('data/games.csv')
df_games=pd.merge(df_games, load_stadium_data(),left_on='gameId', right_on='old_game_id', how='left')
df_games=pd.merge(df_games, load_weather_data(),on='gameId', how='left')

# feed play data thru feature_engineering process
df_plays = feature_engineering(pd.read_csv('data/plays.csv'))

# load ftn


2022 done.
Downcasting floats.


In [4]:
%%capture
ftn_merged = load_ftn()

Add external data, recover merged ID's from 'get_merge_pre' (i.e., product of half of aggregate_data)

In [5]:
merged_id_df = pd.read_csv('data/merged_ids.csv')
xp_fname = os.path.join(root_dir, "data/team_xpass_ratios.csv")
pr_fname = os.path.join(root_dir, "data/team_pass_rates.csv")
cf_fname = os.path.join(root_dir, "data/coverages_faced.csv")
cu_fname = os.path.join(root_dir, 'data/coverages_used.csv')
xp_df = pd.read_csv(xp_fname).drop(columns='Unnamed: 0')
pr_df = pd.read_csv(pr_fname).drop(columns='Unnamed: 0')

In [6]:
xp_df.head(3)

,gameId,playId,off_xpass,def_xpass
0,2022091110,347,0.58,0.58
1,2022091110,400,0.58,0.58
2,2022091110,729,0.58,0.58


In [7]:
xp_df.isna().sum()

gameId       0
playId       0
off_xpass    0
def_xpass    0
dtype: int64

#### Merge in week, team info to our base dataframe

In [8]:
# add week info
merged_base = merged_id_df.merge(df_games[['gameId','week']].drop_duplicates(),how='left',on=['gameId'])

# add in team info
merged_base = merged_base.merge(df_plays[['gameId','playId',
                                          'possessionTeam','defensiveTeam']].drop_duplicates(),
                                how='left',on=['gameId','playId'])

In [36]:
merged_base.head(3)

,gameId,playId,week,possessionTeam,defensiveTeam
0,2022090800,56,1,BUF,LA
1,2022090800,80,1,BUF,LA
2,2022090800,101,1,BUF,LA


#### Define ID's for each segment of the pre- and post- week 1 split:

In [10]:
w1_ids = merged_base[merged_base['week'] == 1][['gameId','playId','week','possessionTeam','defensiveTeam']]
w2_on_ids = merged_base[merged_base['week'] > 1][['gameId','playId','week','possessionTeam','defensiveTeam']]

#### Import 2021 data

In [11]:
cov_21 = pd.read_csv('data_21/cov_21.csv')
team_pr_21 = pd.read_csv('data_21/team_pr_21.csv')

# Calculate team pass rates

Note: we use team pass rate from last year as xpass for week 1 (due to 2021 data incompleteness)

In [18]:
team_pr_21.head(1)

,possessionTeam,pass_rate_def,pass_rate_off
0,ARI,0.573372,0.573464


In [19]:
w1_ids.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam
0,2022090800,56,1,BUF,LA


#### Merge in '21 pass ratios to '22 week one df

We do offensive and defensive rates separately

In [20]:
# merge defensive pass rates for '21 into week 1
w1_pr = w1_ids.merge(team_pr_21.drop(columns=['pass_rate_off']).rename(columns={'possessionTeam':'defensiveTeam'}),
                     on='defensiveTeam',how='left')

# offensive
w1_pr = w1_pr.merge(team_pr_21.drop(columns=['pass_rate_def']),on='possessionTeam',how='left')

We bake in xpass here because it's just going to replicate pass_rate_off/def for week 1

In [24]:
w1_pr['off_xpass'] = w1_pr['pass_rate_off'].copy()
w1_pr['def_xpass'] = w1_pr['pass_rate_def'].copy()

In [25]:
w1_pr.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627


# Calculate coverage data

Here we just use defensive rates for simplicity, can revisit later

In [27]:
# subset to only defensive features, rename features
cov_def = cov_21[[x for x in cov_21.columns if '_off' not in x]].rename(columns={'possessionTeam':'defensiveTeam'})
cov_def = cov_def.rename(columns={'cover_2_def':'Cover-2_def','cover_0_def':'Cover-0_def'})

#merge into running dataframe
w1_merged = w1_pr.merge(cov_def,how='left',on='defensiveTeam')

In [31]:
w1_merged.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass,Man_def,Zone_def,Cover-0_def,cover_1_def,Cover-2_def,cover_3_def,Quarters_def,cover_6_def,Other_def
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627,7.117647,37.117647,0.705882,6.352941,0.647059,20.823529,7.647059,7.705882,0.352941


## Integrate '22 coverage data (i.e., for week 2 on)

In [32]:
w2_on_ids.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam
1950,2022091500,55,2,KC,LAC


Load coverage data, merge:

In [33]:
cu_df = pd.read_csv('data/coverages_used.csv')
cu_df['week'] = cu_df['week'].astype(int)
cu_df.head(1)

,week,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,defensiveTeam
0,2,21.0,2.0,21.0,7.0,4.0,3.0,26.0,4.0,34.0,ARI


In [34]:
cu_w2_on = w2_on_ids.merge(cu_df,how='left',left_on=['defensiveTeam','week'],right_on=['defensiveTeam','week'])

In [35]:
cu_w2_on.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0


### Add pass ratio, expected pass data

In [37]:
df_w2_on = cu_w2_on.merge(pr_df,how='left',on=['gameId','playId'])
df_w2_on = df_w2_on.merge(xp_df,how='left',on=['gameId','playId'])

df_w2_on.head(2)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def,off_xpass,def_xpass
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.625,0.763636,0.686433,0.738111
1,2022091500,76,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.625,0.763636,0.658727,0.738111


In [38]:
w1_merged = w1_merged[df_w2_on.columns]

In [39]:
df_w2_on.week.min()

2

## Integrate week 1-imputed, week 2 onward data

In [57]:
merged_base = pd.concat([w1_merged,df_w2_on],axis=0)

In [58]:
merged_base.sample(3)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def,off_xpass,def_xpass
10832,2022102400,3649,7,CHI,NE,139.0,7.0,142.0,29.0,27.0,10.0,152.0,10.0,202.0,0.504854,0.615176,0.463211,0.574627
11956,2022103008,1350,8,PIT,PHI,131.0,26.0,73.0,83.0,20.0,18.0,92.0,8.0,260.0,0.651332,0.662983,0.649748,0.665599
8413,2022101606,3027,6,NYG,BAL,96.0,39.0,59.0,49.0,32.0,18.0,79.0,17.0,216.0,0.563380,0.707006,0.549100,0.664381


Get cols final for everything to line up later (sans off. coverage feats)

In [59]:
cols_final = ['gameId', 'playId', 'n_offense_backfield', 'n_defense_box',
       'is_no_huddle', 'is_motion', 'pass_rate_off', 'pass_rate_def',
       'off_xpass', 'def_xpass', 'week', 'possessionTeam', 'defensiveTeam',
       'cover_3_def', 'cover_6_def', 'cover_1_def', 'Quarters_def',
       'Cover-2_def', 'Cover-0_def', 'Man_def', 'Other_def', 'Zone_def',
       'off_snaps_lost', 'def_snaps_lost']

In [60]:
inj_fname = os.path.join(root_dir, 'data/snaps_lost_injury.csv')
inj_df = pd.read_csv(inj_fname)

In [61]:
inj_df.head(2)

,team,week,off_snaps_lost,def_snaps_lost
0,ARI,2,21.0,0.0
1,ARI,3,0.0,8.5


In [62]:
# merge in offensive snaps
merged_base = merged_base.merge(inj_df.drop(columns=['def_snaps_lost']),how='left',
                   left_on=['possessionTeam','week'], right_on=['team','week']).drop(columns=['team'])

# merge in defensive snaps
merged_base = merged_base.merge(inj_df.drop(columns=['off_snaps_lost']),how='left',
                   left_on=['defensiveTeam','week'], right_on=['team','week']).drop(columns=['team'])

# TODO: Figure out why NA's for injury for these weeks

In [66]:
merged_base[(merged_base['off_snaps_lost'].isna()) & (merged_base['week'] > 1)]

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def,off_xpass,def_xpass,off_snaps_lost,def_snaps_lost
2191,2022091801,218,2,CLE,NYJ,23.0,2.0,6.0,15.0,1.0,1.0,7.0,0.0,41.0,0.492308,0.640000,0.494254,0.628774,NaN,0.0
2192,2022091801,247,2,CLE,NYJ,23.0,2.0,6.0,15.0,1.0,1.0,7.0,0.0,41.0,0.492308,0.640000,0.474027,0.628774,NaN,0.0
2193,2022091801,271,2,CLE,NYJ,23.0,2.0,6.0,15.0,1.0,1.0,7.0,0.0,41.0,0.492308,0.640000,0.470012,0.661487,NaN,0.0
2194,2022091801,292,2,CLE,NYJ,23.0,2.0,6.0,15.0,1.0,1.0,7.0,0.0,41.0,0.492308,0.640000,0.483916,0.661487,NaN,0.0
2195,2022091801,313,2,CLE,NYJ,23.0,2.0,6.0,15.0,1.0,1.0,7.0,0.0,41.0,0.492308,0.640000,0.480875,0.628774,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14426,2022103012,3403,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,0.693767,0.513021,0.651158,0.473680,NaN,0.5
14427,2022103012,3435,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,0.693767,0.513021,0.671301,0.495843,NaN,0.5
14428,2022103012,3477,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,0.693767,0.513021,0.643485,0.473548,NaN,0.5
14429,2022103012,3498,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,0.693767,0.513021,0.633451,0.492486,NaN,0.5


In [63]:
merged_base.isna().sum()

gameId               0
playId               0
week                 0
possessionTeam       0
defensiveTeam        0
cover_3_def          0
cover_6_def          0
cover_1_def          0
Quarters_def         0
Cover-2_def          0
Cover-0_def          0
Man_def              0
Other_def            0
Zone_def             0
pass_rate_off        0
pass_rate_def        0
off_xpass            0
def_xpass            0
off_snaps_lost    4412
def_snaps_lost    4472
dtype: int64

# TODO: sanity check injury snaps lost process, if sound just fillna w/0

In [ ]:
merged_base[merged_base.off_snaps_lost.isna()].sample(5)

# Integrate FTN, injury data